<a href="https://colab.research.google.com/github/luizleroy/predictive_analytics/blob/develop/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Introduction to Predictive Analysis with Colab and Python**

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/luizleroy/predictive_analytics/blob/develop/new.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Welcome to this Colab! I will get a quick introduction to the Analysis with Colab and Python to Predictive Analysis.

Colab is a Python development environment that runs in the browser using Google Cloud.

In [ ]:
print("Agora, vamos na língua de Camões mesmo, hehehe. Pois, po principal objetivo aqui é criar insumos, 'pedaços' de código mesmo, para utilizar em projetos de P&D.")

Agora, vamos na língua de Camões mesmo, hehehe. Pois, po principal objetivo aqui é criar insumos, 'pedaços' de código mesmo, para utilizar em projetos de P&D.


# Imports

In [2]:
import numpy as np
import pandas as pd
import json


# Code

In [43]:
data = pd.read_json('sample_data/anscombe.json')
X = data['X']
Y = data['Y']
X1 = X[0:11]
X2 = X[11:22]
X3 = X[22:33]
X4 = X[33:42]
columns = ['I','II', 'III', 'IV']
data_para_treinamento_ascombe = pd.DataFrame(columns=columns)
for i in range(11):
  data_para_treinament_ascombe[i] = X1.iloc[[i]]
print(data_para_treinament_ascombe[0])

0    10
Name: X, dtype: int64


In [4]:
# Z = data.transpose()
# print(data)
# print("transpondo...")
# print(Z)